# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [1]:
#This linehas to be run in the terminal
#!pip install openai==1.55.3 tiktoken datasets session-info --quiet

### Imports

In [2]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

c:\Users\arand\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
session_info.show()

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [68]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "d328fb9df91448a3a088ed980f8ada9c",            #Replace it with your credentials
    "AZURE_OPENAI_ENDPOINT": "https://aiforbusiness2024.openai.azure.com/",      #Replace it with your credentials
    "AZURE_OPENAI_APIVERSION": "2024-02-15-preview", #Replace it with your credentials
    "CHATGPT_MODEL": "gpt-4o-mini"             #Replace it with your credentials
}

In [69]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


In [70]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [71]:
creds = json.loads(data)

In [72]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [73]:
chat_model_id = creds["CHATGPT_MODEL"]

In [74]:
chat_model_id

'gpt-4o-mini'

### Utilities

In [22]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo") # changed from gpt-3.5-turbo

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

**(A) Upload and read csv file (2 Marks)**

In [10]:
cs_reviews_df = pd.read_csv("courier-service_reviews.csv")
# Read CSV File Here

In [11]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [9]:
cs_reviews_df.sample(5)
#cs_reviews_df.head()

,id,review,sentiment
96,97,"When it comes to packaging materials, ExpressW...",Positive
127,128,While ExpressWay Logistics' pricing is competi...,Negative
20,21,ExpressWay Logistics consistently delivers exc...,Positive
90,91,ExpressWay Logistics' proactive approach to pr...,Positive
76,77,I had a terrible experience with ExpressWay Lo...,Negative


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [6]:
#Counting the values of positive and negative reviews"___________"
cs_reviews_df["sentiment"].value_counts()

sentiment
Positive    68
Negative    63
Name: count, dtype: int64

In [7]:
cs_reviews_df.shape

(131, 3)

**(C) Split the Dataset (2 Marks)**

In [12]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42,#<- ensures that the splits are the same for every session
    stratify=cs_reviews_df['sentiment'] #<- ensures equal distribution of labels for good measure
)

In [13]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [14]:
columns_to_select = ['review','sentiment']

In [15]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [16]:
gold_examples

'[{"review":"I was impressed by the speed of delivery offered by ExpressWay Logistics, as my package arrived sooner than expected. The prompt service exceeded my expectations and demonstrated their commitment to timely deliveries.However, upon receiving the package, I discovered that an item was missing from the shipment. Despite contacting customer service to address the issue, I encountered difficulty in obtaining a resolution, leaving me dissatisfied with the overall experience.","sentiment":"Negative"},{"review":"I was promised to be updated with courier partner\'s contact details.Nothing happened as said and I faced difficulty finding the contact details and keep a track of my goods.I was scared if I was scammed and thought of raising a support ticket but ended up with a frustrating experience as there was no platform to do so.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' proactive approach to problem-solving sets them apart from other logistics providers. They antic

In [17]:
json.loads(gold_examples)[0]     #Json format

{'review': 'I was impressed by the speed of delivery offered by ExpressWay Logistics, as my package arrived sooner than expected. The prompt service exceeded my expectations and demonstrated their commitment to timely deliveries.However, upon receiving the package, I discovered that an item was missing from the shipment. Despite contacting customer service to address the issue, I encountered difficulty in obtaining a resolution, leaving me dissatisfied with the overall experience.',
 'sentiment': 'Negative'}

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [18]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [44]:
zero_shot_system_message = """You are customer review sentiment classifier assistant. You will be provided with a customer review. Each review will be enclosed by triple backticks.
You need to read the review and then classify it into one of the below 2 types: 
A) 'Positive'
B) 'Negative'
Classify the review in the above 2 sentiments only. DO NOT use any other sentiment. Sentiments like Neutral and Mixed are Unacceptable.
If the review contains both positive and negative sentiment, still classify into one of them and this will be our FINAL SENTIMENT. Now this FINAL SENTIMENT should be the output.
These 2 sentiments - 'Positive' and 'Negative' are the only acceptable answeres.
Compulsary - Answer should only contain one word mentioning FINAL SENTIMENT - either 'Positive' or 'Negative'. DO NOT Explain your answer.
I Repeat, unacceptable responses - Neutral and Mixed. Every review is compulsary to be only and only 'Positive' or 'Negative'
"""
# Write zero shot system message here

**(B) Create Zero Shot Prompt (2 Marks)**

In [45]:
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
# Create zero shot prompt to be input ready for completion function

In [46]:
cs_reviews_df.iloc[0,:]

id                                                           1
review       ExpressWay Logistics' commitment to transparen...
sentiment                                             Positive
Name: 0, dtype: object

In [48]:
input_description = cs_reviews_df.iloc[0,0]

user_input = [
    {
        'role':'user',
        'content': user_message_template.format(courier_service_review = input_description)
    }
]

In [35]:
num_tokens_from_messages(zero_shot_prompt)

203

**(C) Write Few Shot System Message (3 Marks)**

In [36]:
few_shot_system_message = """You are customer review sentiment classifier assistant. You will be provided with a customer review. Each review will be enclosed by triple backticks.
You need to read the review and then classify it into one of the below 2 types: 
A) 'Positive'
B) 'Negative'
Classify the review in the above 2 sentiments only. DO NOT use any other sentiment. Sentiments like Neutral and Mixed are Unacceptable.
If the review contains both positive and negative sentiment, still classify into one of them and this will be our FINAL SENTIMENT. Now this FINAL SENTIMENT should be the output.
These 2 sentiments - 'Positive' and 'Negative' are the only acceptable answeres.
Compulsary - Answer should only contain one word mentioning FINAL SENTIMENT - either 'Positive' or 'Negative'. DO NOT Explain your answer.
I Repeat, unacceptable responses - Neutral and Mixed. Every review is compulsary to be only and only 'Positive' or 'Negative'
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [37]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**(D) Create Examples For Few shot prompte (2 Marks)**

In [26]:
examples = create_examples(cs_examples_df,2)
# Create Examples

In [27]:
json.loads(examples)

[{'review': "I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend! ",
  'sentiment': 'Negative'},
 {'review': "ExpressWay Logistics' customer service representatives are polite but lack the authority to resolve issues effectively. Dealing with them feels like hitting a dead end, with problems often left unresolved.",
  'sentiment': 'Negative'},
 {'review': "I had a sensitive legal document that needed to be delivered to a client across the country, and ExpressWay Logistics' secure courier service ensured that i

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [38]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [39]:
few_shot_prompt = create_prompt(few_shot_system_message,
    examples,
    user_message_template
    )
# Create Few shot prompt

In [40]:
few_shot_prompt

[{'role': 'system',
  'content': "You are customer review sentiment classifier assistant. You will be provided with a customer review. Each review will be enclosed by triple backticks.\nYou need to read the review and then classify it into one of the below 2 types: \nA) 'Positive'\nB) 'Negative'\nClassify the review in the above 2 sentiments only. DO NOT use any other sentiment. Sentiments like Neutral and Mixed are Unacceptable.\nIf the review contains both positive and negative sentiment, still classify into one of them and this will be our FINAL SENTIMENT. Now this FINAL SENTIMENT should be the output.\nThese 2 sentiments - 'Positive' and 'Negative' are the only acceptable answeres.\nCompulsary - Answer should only contain one word mentioning FINAL SENTIMENT - either 'Positive' or 'Negative'. DO NOT Explain your answer.\nI Repeat, unacceptable responses - Neutral and Mixed. Every review is compulsary to be only and only 'Positive' or 'Negative'\n"},
 {'role': 'user',
  'content': "`

In [41]:
num_tokens_from_messages(few_shot_prompt)

465

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [75]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


**(A) Evaluate zero shot prompt (2 Marks)**

In [76]:
evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

1.0

**(B) Evaluate few shot prompt (2 Marks)**

In [77]:
evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

1.0

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [78]:
num_eval_runs = 5

In [79]:
zero_shot_performance = []
few_shot_performance = []

In [80]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

 20%|██        | 1/5 [00:23<01:33, 23.27s/it]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

 40%|████      | 2/5 [00:57<01:28, 29.48s/it]

...

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

 80%|████████  | 4/5 [01:46<00:25, 25.61s/it]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

100%|██████████| 5/5 [02:08<00:00, 25.66s/it]

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                           

**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [81]:
np.array(zero_shot_performance).mean(), np.array(zero_shot_performance).std()
# Calculate for Zero Shot

(1.0, 0.0)

In [82]:
np.array(few_shot_performance).mean(), np.array(few_shot_performance).std()
# Calculate for Few Shot

(1.0, 0.0)

##**Step 5: Observation and Insights and Business perspective (3 Marks)**

( Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified. )


### OBSERVATION(S):
- The micro f1-score for both zero_shot and few_shot prompts are both 1.0 even after 5 evaluations. If this were a machine learning f1-score I would assume that the model is over-fitting. However, since this is not ML model training, but scoring of how each prompt's results compared to the gold-examples, I would say that the prompts are outstanding.  
- Since the zero-shot-prompt has an average performance of 1.0 wich is perfect, we should NOT use the additional tokens needed for the few-shot-prompt. Few-shot is NOT adding any additional value while additional tokens are added cost.  

### INSIGHTS:
**Business Use Case**:  
- These learnings can be useful to as they will give insoght into what their customers are saying about their services. The company states that they are committed to enhance their logistics and courier services and improve seamless connectivity for their customers. How will they know they are achieving this unless they survey responses from their customers, understand those sentiments, then take action where sentiment is negative.  
- Through the application of prompt engineering methodologies and sentiment analysis, this will help them to gain those insights much faster than traditional NLP or more manual methods.  
- In pratical application, these results are useless unless they are in a form that is digestible for stakeholders. As such, this application should be deployed and predictions perhaps transformed into an Excel file and piped to Azure Blob storage for consumption at the least.  

**Breakdown of percentage of positive/negative reviews**:  
There are 131 reviews in the data set. The breakdown is as such:  
Positive    68  
Negative    63  

**How classification can assist Expressway Logistics in addressing the issues identified**:  
I would propose using classification (text-to-label) to label each negative review based on 4 or 5 pre-determined labels such as late, slow, damaged, ect. Even in the case where there are no gold examples of lables, they can be inferred for a small sample, then tasked by the LLM to classify the remaining reviews. These labels can then be analyzed for more actionable insights that speak to customer pain points more so than standard 'positive/negative' sentiment.


